In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        return x

In [ ]:
class DownBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownBlock, self).__init__()
        self.conv_block = ConvBlock(in_channels, out_channels)
        self.pool = nn.MaxPool2d(2)
    
    def forward(self, x):
        x = self.conv_block(x)
        p = self.pool(x)
        return x, p

In [ ]:
class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UpBlock, self).__init__()
        self.upconv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        self.conv_block = ConvBlock(in_channels, out_channels)
    
    def forward(self, x, skip):
        x = self.upconv(x)
        # Match the dimensions of the skip connection
        x = torch.cat([x, skip], dim=1)
        x = self.conv_block(x)
        return x

In [ ]:
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        
        # Encoder
        self.enc1 = DownBlock(in_channels, 64)
        self.enc2 = DownBlock(64, 128)
        self.enc3 = DownBlock(128, 256)
        self.enc4 = DownBlock(256, 512)
        
        # Bottleneck
        self.bottleneck = ConvBlock(512, 1024)
        
        # Decoder
        self.up4 = UpBlock(1024, 512)
        self.up3 = UpBlock(512, 256)
        self.up2 = UpBlock(256, 128)
        self.up1 = UpBlock(128, 64)
        
        # Final convolution to map to the output channels
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)
    
    def forward(self, x):
        # Encoder
        enc1, p1 = self.enc1(x)
        enc2, p2 = self.enc2(p1)
        enc3, p3 = self.enc3(p2)
        enc4, p4 = self.enc4(p3)
        
        # Bottleneck
        bottleneck = self.bottleneck(p4)
        
        # Decoder
        up4 = self.up4(bottleneck, enc4)
        up3 = self.up3(up4, enc3)
        up2 = self.up2(up3, enc2)
        up1 = self.up1(up2, enc1)
        
        # Final output
        out = self.final_conv(up1)
        
        return out

In [ ]:
def iou_score(pred, target, threshold=0.5, num_classes=2):
    """
    Compute the Intersection over Union (IoU) score for segmentation.

    Args:
    - pred (Tensor): Predicted segmentation map (logits or probabilities).
    - target (Tensor): Ground truth segmentation map.
    - threshold (float): Threshold to binarize the prediction (for binary IoU).
    - num_classes (int): Number of classes (for multi-class IoU).
    
    Returns:
    - iou (Tensor): IoU score.
    """
    pred = pred > threshold  # Binarize the prediction (if logits or probabilities)
    iou = torch.zeros(num_classes)  # IoU for each class

    for i in range(num_classes):
        # Create masks for the current class
        intersection = torch.sum((pred == i) & (target == i))  # True positives
        union = torch.sum((pred == i) | (target == i))  # Total predicted or true pixels

        if union != 0:
            iou[i] = intersection / union
        else:
            iou[i] = torch.tensor(float('nan'))  # If no union, IoU is undefined

    return iou.mean()

In [ ]:
def dice_score(pred, target, threshold=0.5, num_classes=2):
    """
    Compute the Dice coefficient for segmentation.

    Args:
    - pred (Tensor): Predicted segmentation map (logits or probabilities).
    - target (Tensor): Ground truth segmentation map.
    - threshold (float): Threshold to binarize the prediction (for binary Dice).
    - num_classes (int): Number of classes (for multi-class Dice).
    
    Returns:
    - dice (Tensor): Dice score.
    """
    pred = pred > threshold  # Binarize the prediction (if logits or probabilities)
    dice = torch.zeros(num_classes)  # Dice score for each class

    for i in range(num_classes):
        # Create masks for the current class
        intersection = torch.sum((pred == i) & (target == i))  # True positives
        total = torch.sum(pred == i) + torch.sum(target == i)  # Predicted or true pixels

        if total != 0:
            dice[i] = 2 * intersection / total
        else:
            dice[i] = torch.tensor(float('nan'))  # If no total, Dice score is undefined

    return dice.mean()